# LAB | Feature Engineering

**Load the data**

In this challenge, we will be working with the same Spaceship Titanic data, like the previous Lab. The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

In [117]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [119]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


**Check the shape of your data**

In [121]:
spaceship.shape

(8693, 14)

**Check for data types**

In [123]:
spaceship.dtypes

PassengerId      object
HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported        bool
dtype: object

**Check for missing values**

In [141]:
spaceship.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6606 entries, 0 to 6605
Data columns (total 25 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Age                        6606 non-null   float64
 1   RoomService                6606 non-null   float64
 2   FoodCourt                  6606 non-null   float64
 3   ShoppingMall               6606 non-null   float64
 4   Spa                        6606 non-null   float64
 5   VRDeck                     6606 non-null   float64
 6   Transported                6606 non-null   bool   
 7   HomePlanet_Earth           6606 non-null   bool   
 8   HomePlanet_Europa          6606 non-null   bool   
 9   HomePlanet_Mars            6606 non-null   bool   
 10  CryoSleep_False            6606 non-null   bool   
 11  CryoSleep_True             6606 non-null   bool   
 12  Cabin_A                    6606 non-null   bool   
 13  Cabin_B                    6606 non-null   bool 

There are multiple strategies to handle missing data

- Removing all rows or all columns containing missing data.
- Filling all missing values with a value (mean in continouos or mode in categorical for example).
- Filling all missing values with an algorithm.

For this exercise, because we have such low amount of null values, we will drop rows containing any missing value. 

In [127]:
spaceship = spaceship.dropna()
spaceship.reset_index(drop=True, inplace=True)

- **Cabin** is too granular - transform it in order to obtain {'A', 'B', 'C', 'D', 'E', 'F', 'G', 'T'}

In [131]:
spaceship['Cabin'] = spaceship['Cabin'].apply(lambda x: x.split('/')[0] if isinstance(x, str) else np.nan)

In [133]:
print(spaceship["Cabin"].unique())

['B' 'F' 'A' 'G' 'E' 'C' 'D' 'T']


- Drop PassengerId and Name

In [135]:
spaceship = spaceship.drop(columns=['Name'])
spaceship = spaceship.drop(columns=['PassengerId'])

- For non-numerical columns, do dummies.

In [139]:
spaceship = pd.get_dummies(spaceship, columns=['HomePlanet','CryoSleep','Cabin','Destination','VIP'])

In [143]:
spaceship_numerical = spaceship.select_dtypes(include=['number'])

In [145]:
normalizer = MinMaxScaler()

In [147]:
spaceship_numerical_norm = normalizer.fit_transform(spaceship_numerical)

In [149]:
spaceship_numerical_norm_df = pd.DataFrame(spaceship_numerical_norm, columns=spaceship_numerical.columns)

In [151]:
numerical_cols = spaceship.select_dtypes(include=['number']).columns
spaceship = spaceship.drop(columns=numerical_cols)

In [153]:
spaceship_combined = pd.concat([spaceship, spaceship_numerical_scaled_df], axis=1)

In [155]:
boolean_cols = spaceship_combined.select_dtypes(include=['bool']).columns
spaceship_combined[boolean_cols] = spaceship_combined[boolean_cols].astype(int)

In [188]:
spaceship_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6606 entries, 0 to 6605
Data columns (total 25 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Transported                6606 non-null   int32  
 1   HomePlanet_Earth           6606 non-null   int32  
 2   HomePlanet_Europa          6606 non-null   int32  
 3   HomePlanet_Mars            6606 non-null   int32  
 4   CryoSleep_False            6606 non-null   int32  
 5   CryoSleep_True             6606 non-null   int32  
 6   Cabin_A                    6606 non-null   int32  
 7   Cabin_B                    6606 non-null   int32  
 8   Cabin_C                    6606 non-null   int32  
 9   Cabin_D                    6606 non-null   int32  
 10  Cabin_E                    6606 non-null   int32  
 11  Cabin_F                    6606 non-null   int32  
 12  Cabin_G                    6606 non-null   int32  
 13  Cabin_T                    6606 non-null   int32

**Perform Train Test Split**

In [159]:
# X-y split; features = X, target = y
features = spaceship_combined.drop(columns = ["Transported"])
target = spaceship_combined["Transported"]

In [163]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.20, random_state=1)

**Model Selection**

In this exercise we will be using **KNN** as our predictive model.

In [165]:
knn = KNeighborsRegressor()

In [167]:
knn.fit(X_train, y_train)

KNeighborsRegressor()

- Evaluate your model's performance. Comment it

In [172]:
print(f"The accuracy of the model is {knn.score(X_test, y_test)*100: .2f}%")

The accuracy of the model is  32.63%


Accuracy dropped from 76.55% when using only numerical columns to 32.63% when using all columns.